In [39]:
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 24 21:26:07 2018

@author: Kevin
"""

import pandas as pd
import numpy as np
import time
import pulp  

###讀檔###
Operation_sequence=pd.read_excel("FJSPOperation_sequence.xlsx")
Processing_time=pd.read_excel("FJSPProcessing_time.xlsx")

ops = Operation_sequence.values
pt = Processing_time.values



In [40]:
ops

array([[ 1.,  5.,  4.,  2.,  3.],
       [ 2.,  1.,  4.,  2., nan],
       [ 3.,  3.,  5.,  1.,  4.],
       [ 4.,  3.,  2.,  5.,  4.],
       [ 5.,  4.,  6., nan, nan]])

In [41]:
type(ops)

numpy.ndarray

In [42]:
#ops = ops.astype(int)

In [43]:
ops

array([[ 1.,  5.,  4.,  2.,  3.],
       [ 2.,  1.,  4.,  2., nan],
       [ 3.,  3.,  5.,  1.,  4.],
       [ 4.,  3.,  2.,  5.,  4.],
       [ 5.,  4.,  6., nan, nan]])

In [44]:
pt

array([[1, 3, nan, 68.0, 58.0, nan, 55.0],
       [1, 5, nan, 84.0, 74.0, ',', 85.0],
       [2, 2, 40.0, 10.0, nan, 7, nan],
       [2, 3, 88.0, 15.0, nan, 13, nan],
       [3, 5, 5.0, nan, 18.0, 40, 17.0],
       [3, 4, 10.0, nan, 52.0, 47, 42.0],
       [4, 4, 71.0, 87.0, 14.0, 11, 64.0],
       [4, 2, 74.0, 88.0, 22.0, 30, 65.0],
       [5, 2, 18.0, nan, 56.0, 66, nan],
       [5, 3, 22.0, nan, 64.0, 80, nan],
       [6, 2, nan, nan, nan, nan, 74.0],
       [6, 1, nan, nan, nan, nan, 88.0]], dtype=object)

In [45]:
###定義問題###
model = pulp.LpProblem("MIN_makespan", pulp.LpMinimize)

###決策變數###
Cmax = pulp.LpVariable('Cmax',lowBound = 0, cat='Continuous')



In [46]:
Ci = pulp.LpVariable.dicts("start_time",
                           ((i) for i in range(1,6)),
                           lowBound=0,
                           cat='Continuous')

In [47]:
Ci

{1: start_time_1,
 2: start_time_2,
 3: start_time_3,
 4: start_time_4,
 5: start_time_5}

In [48]:
#i=job,j=operation,k=machine
st = pulp.LpVariable.dicts("start_time",
                           ((i, j, k) for i in range(1,6) for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0] for k in list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])),
                           lowBound=0,
                           cat='Continuous')

st

{(1, 5.0, 2): start_time_(1,_5.0,_2),
 (1, 5.0, 3): start_time_(1,_5.0,_3),
 (1, 4.0, 4): start_time_(1,_4.0,_4),
 (1, 4.0, 2): start_time_(1,_4.0,_2),
 (1, 2.0, 2): start_time_(1,_2.0,_2),
 (1, 2.0, 3): start_time_(1,_2.0,_3),
 (1, 3.0, 5): start_time_(1,_3.0,_5),
 (1, 3.0, 4): start_time_(1,_3.0,_4),
 (2, 1.0, 3): start_time_(2,_1.0,_3),
 (2, 1.0, 5): start_time_(2,_1.0,_5),
 (2, 4.0, 4): start_time_(2,_4.0,_4),
 (2, 4.0, 2): start_time_(2,_4.0,_2),
 (2, 2.0, 2): start_time_(2,_2.0,_2),
 (2, 2.0, 3): start_time_(2,_2.0,_3),
 (3, 3.0, 5): start_time_(3,_3.0,_5),
 (3, 3.0, 4): start_time_(3,_3.0,_4),
 (3, 5.0, 2): start_time_(3,_5.0,_2),
 (3, 5.0, 3): start_time_(3,_5.0,_3),
 (3, 1.0, 3): start_time_(3,_1.0,_3),
 (3, 1.0, 5): start_time_(3,_1.0,_5),
 (3, 4.0, 4): start_time_(3,_4.0,_4),
 (3, 4.0, 2): start_time_(3,_4.0,_2),
 (4, 3.0, 5): start_time_(4,_3.0,_5),
 (4, 3.0, 4): start_time_(4,_3.0,_4),
 (4, 2.0, 2): start_time_(4,_2.0,_2),
 (4, 2.0, 3): start_time_(4,_2.0,_3),
 (4, 5.0, 2)

In [49]:
ct = pulp.LpVariable.dicts("completed_time",
                           ((i, j, k) for i in range(1,6) for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0] for k in list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])),
                           lowBound=0,
                           cat='Continuous')
ct

{(1, 5.0, 2): completed_time_(1,_5.0,_2),
 (1, 5.0, 3): completed_time_(1,_5.0,_3),
 (1, 4.0, 4): completed_time_(1,_4.0,_4),
 (1, 4.0, 2): completed_time_(1,_4.0,_2),
 (1, 2.0, 2): completed_time_(1,_2.0,_2),
 (1, 2.0, 3): completed_time_(1,_2.0,_3),
 (1, 3.0, 5): completed_time_(1,_3.0,_5),
 (1, 3.0, 4): completed_time_(1,_3.0,_4),
 (2, 1.0, 3): completed_time_(2,_1.0,_3),
 (2, 1.0, 5): completed_time_(2,_1.0,_5),
 (2, 4.0, 4): completed_time_(2,_4.0,_4),
 (2, 4.0, 2): completed_time_(2,_4.0,_2),
 (2, 2.0, 2): completed_time_(2,_2.0,_2),
 (2, 2.0, 3): completed_time_(2,_2.0,_3),
 (3, 3.0, 5): completed_time_(3,_3.0,_5),
 (3, 3.0, 4): completed_time_(3,_3.0,_4),
 (3, 5.0, 2): completed_time_(3,_5.0,_2),
 (3, 5.0, 3): completed_time_(3,_5.0,_3),
 (3, 1.0, 3): completed_time_(3,_1.0,_3),
 (3, 1.0, 5): completed_time_(3,_1.0,_5),
 (3, 4.0, 4): completed_time_(3,_4.0,_4),
 (3, 4.0, 2): completed_time_(3,_4.0,_2),
 (4, 3.0, 5): completed_time_(4,_3.0,_5),
 (4, 3.0, 4): completed_time_(4,_3

In [50]:

x = pulp.LpVariable.dicts("X_binary_var",
                          ((i, j, k) for i in range(1,6) for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0] for k in list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])),
                          lowBound=0,
                          cat='Binary')
x

{(1, 5.0, 2): X_binary_var_(1,_5.0,_2),
 (1, 5.0, 3): X_binary_var_(1,_5.0,_3),
 (1, 4.0, 4): X_binary_var_(1,_4.0,_4),
 (1, 4.0, 2): X_binary_var_(1,_4.0,_2),
 (1, 2.0, 2): X_binary_var_(1,_2.0,_2),
 (1, 2.0, 3): X_binary_var_(1,_2.0,_3),
 (1, 3.0, 5): X_binary_var_(1,_3.0,_5),
 (1, 3.0, 4): X_binary_var_(1,_3.0,_4),
 (2, 1.0, 3): X_binary_var_(2,_1.0,_3),
 (2, 1.0, 5): X_binary_var_(2,_1.0,_5),
 (2, 4.0, 4): X_binary_var_(2,_4.0,_4),
 (2, 4.0, 2): X_binary_var_(2,_4.0,_2),
 (2, 2.0, 2): X_binary_var_(2,_2.0,_2),
 (2, 2.0, 3): X_binary_var_(2,_2.0,_3),
 (3, 3.0, 5): X_binary_var_(3,_3.0,_5),
 (3, 3.0, 4): X_binary_var_(3,_3.0,_4),
 (3, 5.0, 2): X_binary_var_(3,_5.0,_2),
 (3, 5.0, 3): X_binary_var_(3,_5.0,_3),
 (3, 1.0, 3): X_binary_var_(3,_1.0,_3),
 (3, 1.0, 5): X_binary_var_(3,_1.0,_5),
 (3, 4.0, 4): X_binary_var_(3,_4.0,_4),
 (3, 4.0, 2): X_binary_var_(3,_4.0,_2),
 (4, 3.0, 5): X_binary_var_(4,_3.0,_5),
 (4, 3.0, 4): X_binary_var_(4,_3.0,_4),
 (4, 2.0, 2): X_binary_var_(4,_2.0,_2),


In [51]:
y = pulp.LpVariable.dicts("Y_binary_var",
                          ((i, j, l, m,k) for i in range(1,6) for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]  for l in range(1,6) for m in [ops[l-1][a] for a in range(1,5) if ops[l-1][a]!=0]  for k in list(set(list(Processing_time.loc[(Processing_time['Operation']==j),'machine']))&set(list(Processing_time.loc[(Processing_time['Operation']==m),'machine']))) if i < l ),
                          lowBound=0,
                          cat='Binary')
y

{(1, 5.0, 2, 1.0, 3): Y_binary_var_(1,_5.0,_2,_1.0,_3),
 (1, 5.0, 2, 4.0, 2): Y_binary_var_(1,_5.0,_2,_4.0,_2),
 (1, 5.0, 2, 2.0, 2): Y_binary_var_(1,_5.0,_2,_2.0,_2),
 (1, 5.0, 2, 2.0, 3): Y_binary_var_(1,_5.0,_2,_2.0,_3),
 (1, 5.0, 3, 5.0, 2): Y_binary_var_(1,_5.0,_3,_5.0,_2),
 (1, 5.0, 3, 5.0, 3): Y_binary_var_(1,_5.0,_3,_5.0,_3),
 (1, 5.0, 3, 1.0, 3): Y_binary_var_(1,_5.0,_3,_1.0,_3),
 (1, 5.0, 3, 4.0, 2): Y_binary_var_(1,_5.0,_3,_4.0,_2),
 (1, 5.0, 4, 2.0, 2): Y_binary_var_(1,_5.0,_4,_2.0,_2),
 (1, 5.0, 4, 2.0, 3): Y_binary_var_(1,_5.0,_4,_2.0,_3),
 (1, 5.0, 4, 5.0, 2): Y_binary_var_(1,_5.0,_4,_5.0,_2),
 (1, 5.0, 4, 5.0, 3): Y_binary_var_(1,_5.0,_4,_5.0,_3),
 (1, 5.0, 4, 4.0, 2): Y_binary_var_(1,_5.0,_4,_4.0,_2),
 (1, 5.0, 5, 4.0, 2): Y_binary_var_(1,_5.0,_5,_4.0,_2),
 (1, 5.0, 5, 6.0, 2): Y_binary_var_(1,_5.0,_5,_6.0,_2),
 (1, 4.0, 2, 4.0, 2): Y_binary_var_(1,_4.0,_2,_4.0,_2),
 (1, 4.0, 2, 4.0, 4): Y_binary_var_(1,_4.0,_2,_4.0,_4),
 (1, 4.0, 2, 2.0, 2): Y_binary_var_(1,_4.0,_2,_2

In [52]:
model += Cmax
model

MIN_makespan:
MINIMIZE
1*Cmax + 0
VARIABLES
Cmax Continuous

In [55]:
for i in range(1,6):
    for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]:
        print('j' + str(j))
        ma_list=list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])
        print(ma_list)
        model += pulp.lpSum(x[i,j,ma] for ma in ma_list)==1

ma_list

j5.0
[2, 3]
j4.0
[4, 2]
j2.0
[2, 3]
j3.0
[5, 4]
j1.0
[3, 5]
j4.0
[4, 2]
j2.0
[2, 3]
jnan
[]
j3.0
[5, 4]
j5.0
[2, 3]
j1.0
[3, 5]
j4.0
[4, 2]
j3.0
[5, 4]
j2.0
[2, 3]
j5.0
[2, 3]
j4.0
[4, 2]
j4.0
[4, 2]
j6.0
[2, 1]
jnan
[]
jnan
[]


[]

In [56]:
model

MIN_makespan:
MINIMIZE
1*Cmax + 0
SUBJECT TO
_C1: X_binary_var_(1,_5.0,_2) + X_binary_var_(1,_5.0,_3) = 1

_C2: X_binary_var_(1,_4.0,_2) + X_binary_var_(1,_4.0,_4) = 1

_C3: X_binary_var_(1,_2.0,_2) + X_binary_var_(1,_2.0,_3) = 1

_C4: X_binary_var_(1,_3.0,_4) + X_binary_var_(1,_3.0,_5) = 1

_C5: X_binary_var_(2,_1.0,_3) + X_binary_var_(2,_1.0,_5) = 1

_C6: X_binary_var_(2,_4.0,_2) + X_binary_var_(2,_4.0,_4) = 1

_C7: X_binary_var_(2,_2.0,_2) + X_binary_var_(2,_2.0,_3) = 1

_C8:0 = 1

_C9: X_binary_var_(3,_3.0,_4) + X_binary_var_(3,_3.0,_5) = 1

_C10: X_binary_var_(3,_5.0,_2) + X_binary_var_(3,_5.0,_3) = 1

_C11: X_binary_var_(3,_1.0,_3) + X_binary_var_(3,_1.0,_5) = 1

_C12: X_binary_var_(3,_4.0,_2) + X_binary_var_(3,_4.0,_4) = 1

_C13: X_binary_var_(4,_3.0,_4) + X_binary_var_(4,_3.0,_5) = 1

_C14: X_binary_var_(4,_2.0,_2) + X_binary_var_(4,_2.0,_3) = 1

_C15: X_binary_var_(4,_5.0,_2) + X_binary_var_(4,_5.0,_3) = 1

_C16: X_binary_var_(4,_4.0,_2) + X_binary_var_(4,_4.0,_4) = 1

_C17: X

In [57]:

for i in range(1,6):
    for j in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]:
        ma_list=list(Processing_time.loc[(Processing_time['Operation']==j),'machine'])
        for k in ma_list:
            ptime=int(Processing_time.loc[(Processing_time['Operation']==j)&(Processing_time['machine']==k),['J%d'%i]].iloc[0])
            I = i
            J = j
            K = k
            model += st[I,J,K] + ct[I,J,K] <=x[I,J,K]*1000
            model += ct[I,J,K] >= st[I,J,K] +ptime-(1-x[I,J,K])*1000

In [58]:
model

MIN_makespan:
MINIMIZE
1*Cmax + 0
SUBJECT TO
_C1: X_binary_var_(1,_5.0,_2) + X_binary_var_(1,_5.0,_3) = 1

_C2: X_binary_var_(1,_4.0,_2) + X_binary_var_(1,_4.0,_4) = 1

_C3: X_binary_var_(1,_2.0,_2) + X_binary_var_(1,_2.0,_3) = 1

_C4: X_binary_var_(1,_3.0,_4) + X_binary_var_(1,_3.0,_5) = 1

_C5: X_binary_var_(2,_1.0,_3) + X_binary_var_(2,_1.0,_5) = 1

_C6: X_binary_var_(2,_4.0,_2) + X_binary_var_(2,_4.0,_4) = 1

_C7: X_binary_var_(2,_2.0,_2) + X_binary_var_(2,_2.0,_3) = 1

_C8:0 = 1

_C9: X_binary_var_(3,_3.0,_4) + X_binary_var_(3,_3.0,_5) = 1

_C10: X_binary_var_(3,_5.0,_2) + X_binary_var_(3,_5.0,_3) = 1

_C11: X_binary_var_(3,_1.0,_3) + X_binary_var_(3,_1.0,_5) = 1

_C12: X_binary_var_(3,_4.0,_2) + X_binary_var_(3,_4.0,_4) = 1

_C13: X_binary_var_(4,_3.0,_4) + X_binary_var_(4,_3.0,_5) = 1

_C14: X_binary_var_(4,_2.0,_2) + X_binary_var_(4,_2.0,_3) = 1

_C15: X_binary_var_(4,_5.0,_2) + X_binary_var_(4,_5.0,_3) = 1

_C16: X_binary_var_(4,_4.0,_2) + X_binary_var_(4,_4.0,_4) = 1

_C17: X

In [59]:




#T = pulp.LpVariable.dicts("completed_time",
#                          ((i, j, k) for i in range(1,6) for j in range(1,7) for k in range(1,6)),
#                          lowBound=0,
#                          cat='Continuous')



###目標式###

#model += Cmax

###限制式###



for i in range(1,6):
    for l in range(1,6):
        if i<l:
            for o1 in [ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]:
                ma1_list=list(Processing_time.loc[(Processing_time['Operation']==o1)&(Processing_time.loc[:]['J%d'%i]!=0),'machine'])
                for o2 in [ops[l-1][o] for o in range(1,5) if ops[l-1][o]!=0]:
                    ma2_list=list(Processing_time.loc[(Processing_time['Operation']==o2)&(Processing_time.loc[:]['J%d'%l]!=0),'machine'])
                    mach_set=list(set(ma1_list)&set(ma2_list))
                    if len(mach_set)>0:
                        for k in mach_set:
                            model += st[i,o1,k]>=ct[l,o2,k]-y[i,o1,l,o2,k]*1000
                            model += st[l,o2,k]>=ct[i,o1,k]-(1-y[i,o1,l,o2,k])*1000

for i in range(1,6):
    op_num=[ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]
    for j in range(1,len(op_num)):
        preop_num=op_num[j-1]
        curop_num=op_num[j]
        prema_list=list(Processing_time.loc[(Processing_time['Operation']==preop_num)&(Processing_time.loc[:]['J%d'%i]!=0),'machine'])
        curma_list=list(Processing_time.loc[(Processing_time['Operation']==curop_num)&(Processing_time.loc[:]['J%d'%i]!=0),'machine'])
        model += pulp.lpSum(st[i,curop_num,ma1] for ma1 in curma_list) >= pulp.lpSum(ct[i,preop_num,ma2] for ma2 in prema_list)


for i in range(1,6):
    op_list=[ops[i-1][o] for o in range(1,5) if ops[i-1][o]!=0]
    last_op=op_list[-1]
    model +=Ci[i]>=pulp.lpSum(ct[i,last_op,k] for k in list(Processing_time.loc[(Processing_time['Operation']==last_op),'machine']))
    model += Cmax >= Ci[i]


start_time = time.time()

model.solve()

pulp.LpStatus[model.status]     

for var in ct:
    var_value = ct[var].varValue
    print( var[0],var[1],var[2] ,var_value)

total_cost = pulp.value(model.objective)
print ('min cost:',total_cost)
print('the elapsed time:%s'% (time.time() - start_time))


1 5.0 2 0.0
1 5.0 3 0.0
1 4.0 4 0.0
1 4.0 2 0.0
1 2.0 2 0.0
1 2.0 3 0.0
1 3.0 5 0.0
1 3.0 4 0.0
2 1.0 3 0.0
2 1.0 5 0.0
2 4.0 4 0.0
2 4.0 2 0.0
2 2.0 2 0.0
2 2.0 3 0.0
3 3.0 5 0.0
3 3.0 4 0.0
3 5.0 2 0.0
3 5.0 3 0.0
3 1.0 3 0.0
3 1.0 5 0.0
3 4.0 4 0.0
3 4.0 2 0.0
4 3.0 5 0.0
4 3.0 4 0.0
4 2.0 2 0.0
4 2.0 3 0.0
4 5.0 2 0.0
4 5.0 3 0.0
4 4.0 4 0.0
4 4.0 2 0.0
5 4.0 4 0.0
5 4.0 2 0.0
5 6.0 2 0.0
5 6.0 1 0.0
min cost: 0.0
the elapsed time:0.4587724208831787


In [ ]:
ct[1,5,2].varValue
# for i in st:
#     print(st[i].varValue)
#     print(st[1,5,2].varValue)
#     print( i[0],i[1],i[2])

In [ ]:
for i in x:
    print(i[0],i[1],i[2],'----',x[i].varValue)


In [ ]:
import plotly
plotly.tools.set_credentials_file(username='swlkevin', api_key='3lvc4tLz4WFTuPUknM39')

In [ ]:
'''--------plot gantt chart-------'''
import pandas as pd
import plotly.plotly as py
import plotly.figure_factory as ff
import datetime
from operator import itemgetter


m_keys=[j+1 for j in range(6)]
j_keys=[j for j in range(5)]
index_record=[]
j_record={}

for var in x:
    if x[var].varValue==1:
        start_time=str(datetime.timedelta(seconds=st[var[0],var[1],var[2]].varValue)) # convert seconds to hours, minutes and seconds
        end_time=str(datetime.timedelta(seconds=ct[var[0],var[1],var[2]].varValue))
        j_record[(var[0],var[1],var[2])]=[start_time,end_time]
        index_record.append([var[0],var[1],var[2]])
index_record.sort(key=itemgetter(2))
        

df=[]
for i in index_record:
    df.append(dict(Task='Machine %s'%(i[2]), Start='2018-11-26 %s'%(str(j_record[(i[0],i[1],i[2])][0])), Finish='2018-11-26 %s'%(str(j_record[(i[0],i[1],i[2])][1])),Resource='Job %s'%(i[0])))
    
fig = ff.create_gantt(df, index_col='Resource', show_colorbar=True, group_tasks=True, showgrid_x=True, title='Flexible_Job shop Schedule')
py.iplot(fig, filename='flexible_job_shop_scheduling', world_readable=True)

In [ ]:
index_record